### Tool Calling
- Setting Up OpenAI API
- Create a custom tools
- Invoke Tool Call
- Manage Message Flow

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
llm.invoke(
    "How will the weather be in bangalore today?"
)

#### Custom Tools

In [ ]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["bangalore"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "bangalore" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "bangalore" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."

tools = [get_weather, check_seating_availability]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
result = llm_with_tools.invoke("How will the weather be in bangalore today?")
result

In [ ]:
result.tool_calls

In [ ]:
result = llm_with_tools.invoke(
    "How will the weather be in bangalore today? Do you still have seats outdoor available?"
)
result

In [ ]:
result.tool_calls

In [ ]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

#### Maintaining Message History

In [27]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in bangalore today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [28]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [29]:
messages

[HumanMessage(content='How will the weather be in bangalore today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8mIVIfdsitDw2OxtSHcwVBol', 'function': {'arguments': '{"location": "bangalore"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_1WHuUUttiKS3qQiRkp8QUHK7', 'function': {'arguments': '{"location": "bangalore", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 92, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C4jDCNGIbSk39xuERR9sZ5KZCqy66', 'service_tier': 'default'

In [32]:
res = llm_with_tools.invoke(messages)
res

AIMessage(content='The weather in Bangalore today is 15 degrees Celsius and cloudy. And yes, we still have seats available outdoors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 177, 'total_tokens': 201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C4jDWFWvJ9DHeuR4fQiZHhjO7hO9B', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f4a6e0d5-4cf2-41f0-b0be-0d30631e971b-0', usage_metadata={'input_tokens': 177, 'output_tokens': 24, 'total_tokens': 201, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [33]:
res.content

'The weather in Bangalore today is 15 degrees Celsius and cloudy. And yes, we still have seats available outdoors.'